In [1]:
import swiftest
from astroquery.jplhorizons import Horizons
import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
sim_gr = swiftest.Simulation(param_file="grsim/param.gr.in", output_file_name="bin.gr.nc")
sim_gr.add_solar_system_body(["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"])

Reading Swiftest file /home/daminton/git_debug/swiftest/examples/helio_gr_test/grsim/param.gr.in
Creating the Sun as a central body
Fetching ephemerides data for Mercury from JPL/Horizons
Fetching ephemerides data for Venus from JPL/Horizons
Fetching ephemerides data for Earth from JPL/Horizons
Fetching ephemerides data for Mars from JPL/Horizons
Fetching ephemerides data for Jupiter from JPL/Horizons
Fetching ephemerides data for Saturn from JPL/Horizons
Fetching ephemerides data for Uranus from JPL/Horizons
Fetching ephemerides data for Neptune from JPL/Horizons
Writing initial conditions to file /home/daminton/git_debug/swiftest/examples/helio_gr_test/grsim/init_cond.nc
Writing parameter inputs to file /home/daminton/git_debug/swiftest/examples/helio_gr_test/grsim/param.gr.in


<xarray.Dataset>
Dimensions:        (name: 9, time: 1)
Coordinates:
  * name           (name) <U32 'Sun' 'Mercury' 'Venus' ... 'Uranus' 'Neptune'
  * time           (time) float64 0.0
Data variables: (12/14)
    particle_type  (name) <U32 'Central Body' 'Massive Body' ... 'Massive Body'
    id             (name) int64 0 1 2 3 4 5 6 7 8
    a              (time, name) float64 nan 0.3871 0.7233 ... 9.532 19.24 30.04
    e              (time, name) float64 nan 0.2056 0.006718 ... 0.04796 0.008956
    inc            (time, name) float64 nan 7.003 3.394 ... 2.488 0.773 1.771
    capom          (time, name) float64 nan 48.3 76.6 ... 113.6 74.01 131.8
    ...             ...
    Gmass          (time, name) float64 39.48 6.554e-06 ... 0.001724 0.002034
    radius         (time, name) float64 0.00465 1.631e-05 ... 0.0001646
    j2rp2          (time, name) float64 4.754e-12 nan nan nan ... nan nan nan
    j4rp4          (time, name) float64 -2.247e-18 nan nan nan ... nan nan nan
    ntp            (time) int64 0
    npl            (time) int64 8

In [3]:
sim_nogr = swiftest.Simulation(param_file="nogrsim/param.nogr.in", output_file_name="bin.nogr.nc")
sim_nogr.add_solar_system_body(["Sun","Mercury","Venus","Earth","Mars","Jupiter","Saturn","Uranus","Neptune"])

Reading Swiftest file /home/daminton/git_debug/swiftest/examples/helio_gr_test/nogrsim/param.nogr.in
Creating the Sun as a central body
Fetching ephemerides data for Mercury from JPL/Horizons
Fetching ephemerides data for Venus from JPL/Horizons
Fetching ephemerides data for Earth from JPL/Horizons
Fetching ephemerides data for Mars from JPL/Horizons
Fetching ephemerides data for Jupiter from JPL/Horizons
Fetching ephemerides data for Saturn from JPL/Horizons
Fetching ephemerides data for Uranus from JPL/Horizons
Fetching ephemerides data for Neptune from JPL/Horizons
Writing initial conditions to file /home/daminton/git_debug/swiftest/examples/helio_gr_test/nogrsim/init_cond.nc
Writing parameter inputs to file /home/daminton/git_debug/swiftest/examples/helio_gr_test/nogrsim/param.nogr.in


<xarray.Dataset>
Dimensions:        (name: 9, time: 1)
Coordinates:
  * name           (name) <U32 'Sun' 'Mercury' 'Venus' ... 'Uranus' 'Neptune'
  * time           (time) float64 0.0
Data variables: (12/14)
    particle_type  (name) <U32 'Central Body' 'Massive Body' ... 'Massive Body'
    id             (name) int64 0 1 2 3 4 5 6 7 8
    a              (time, name) float64 nan 0.3871 0.7233 ... 9.532 19.24 30.04
    e              (time, name) float64 nan 0.2056 0.006718 ... 0.04796 0.008956
    inc            (time, name) float64 nan 7.003 3.394 ... 2.488 0.773 1.771
    capom          (time, name) float64 nan 48.3 76.6 ... 113.6 74.01 131.8
    ...             ...
    Gmass          (time, name) float64 39.48 6.554e-06 ... 0.001724 0.002034
    radius         (time, name) float64 0.00465 1.631e-05 ... 0.0001646
    j2rp2          (time, name) float64 4.754e-12 nan nan nan ... nan nan nan
    j4rp4          (time, name) float64 -2.247e-18 nan nan nan ... nan nan nan
    ntp            (time) int64 0
    npl            (time) int64 8

In [ ]:
tstep_out = 10.0
sim_gr.run(tstop=1000.0, dt=0.005, tstep_out=tstep_out, integrator="helio",general_relativity=True)

In [ ]:
sim_nogr.run(tstop=1000.0, dt=0.005, tstep_out=tstep_out, integrator="helio",general_relativity=False)

In [ ]:
# Get the start and end date of the simulation so we can compare with the real solar system
start_date = sim_gr.ephemeris_date
tstop_d = sim_gr.param['TSTOP'] * sim_gr.param['TU2S'] / swiftest.JD2S

stop_date = (datetime.datetime.fromisoformat(start_date) + datetime.timedelta(days=tstop_d)).isoformat()

In [ ]:
#Get the ephemerides of Mercury for the same timeframe as the simulation
obj = Horizons(id='1', location='@sun',
               epochs={'start':start_date, 'stop':stop_date,
                       'step':'10y'})
el = obj.elements()
t = (el['datetime_jd']-el['datetime_jd'][0]) / 365.25
varpi_obs = el['w'] + el['Omega']

In [ ]:
# Compute the longitude of the periapsis
sim_gr.data['varpi'] = np.mod(sim_gr.data['omega'] + sim_gr.data['capom'],360)
sim_nogr.data['varpi'] = np.mod(sim_nogr.data['omega'] + sim_nogr.data['capom'],360)

In [ ]:
varpisim_gr= sim_gr.data['varpi'].sel(name="Mercury")
varpisim_nogr= sim_nogr.data['varpi'].sel(name="Mercury")
tsim = sim_gr.data['time']

In [ ]:
dvarpi_gr = np.diff(varpisim_gr)  * 3600 * 100 / tstep_out
dvarpi_nogr = np.diff(varpisim_nogr)  * 3600 * 100 / tstep_out
dvarpi_obs = np.diff(varpi_obs) / np.diff(t) * 3600 * 100

In [ ]:
fig, ax = plt.subplots()

ax.plot(t, varpi_obs, label="JPL Horizons",linewidth=2.5)
ax.plot(tsim, varpisim_gr, label="Swiftest helio GR",linewidth=1.5)
ax.plot(tsim, varpisim_nogr, label="Swiftest helio No GR",linewidth=1.5)
ax.set_xlabel('Time (y)')
ax.set_ylabel('Mercury $\\varpi$ (deg)')
ax.legend()
plt.savefig("helio_gr_mercury_precession.png",dpi=300)
print('Mean precession rate for Mercury long. peri. (arcsec/100 y)')
print(f'JPL Horizons         : {np.mean(dvarpi_obs)}')
print(f'Swiftest No GR       : {np.mean(dvarpi_nogr)}')
print(f'Swiftest GR          : {np.mean(dvarpi_gr)}')
print(f'Obs - Swiftest GR    : {np.mean(dvarpi_obs - dvarpi_gr)}')
print(f'Obs - Swiftest No GR : {np.mean(dvarpi_obs - dvarpi_nogr)}')